In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import json
pd.set_option("display.max.columns", None)

In [2]:
# Obtain the list of id's for the webpages to scrape
iedbfile = pd.read_csv('iedb_data_raw.csv', sep=',', error_bad_lines=False, dtype='unicode')
new_header = iedbfile.iloc[0] 
iedbfile = iedbfile[1:] 
iedbfile.columns = new_header
listofIds = iedbfile['T Cell ID'].tolist()
#iedbfile[iedbfile['T Cell ID'] == '3968896']
iedbfile.head(5)

,T Cell ID,Reference ID,Type,PubMed ID,Authors,Journal,Date,Title,Submission ID,Epitope ID,Object Type,Description,Starting Position,Ending Position,Non-peptidic epitope ChEBI ID,Antigen Name,Parent Protein,Parent Protein Accession,Organism Name,Parent Species,Parent Species ID,Epitope Comments,Epitope Relationship,Object Type,Description,Starting Position,Ending Position,Non-peptidic object Accession,Synonyms,Antigen Name,Parent Protein,Organism Name,Parent Organism,Name,Host ID,Geolocation,MHC Types Present,Process Type,Disease State,Disease Stage,Immunogen Epitope Relation,Immunogen Object Type,Immunogen Description,Immunogen Starting Position,Immunogen Ending Position,Non-peptidic Immunogen ChEBI ID,Immunogen Source Molecule Name,Immunogen protein parent Name,Immunogen protein parent Accession,Immunogen Organism Name,Immunogen Organism Species,Immunogen Organism Species Accession,Process Type,Disease State,Disease Stage,Immunogen Epitope Relation,Immunogen Object Type,Immunogen Description,Immunogen Starting Position,Immunogen Ending Position,Non-peptidic Immunogen ChEBI ID,Immunogen Source Molecule Name,Immunogen protein parent Name,Immunogen protein parent Accession,Immunogen Organism Name,Immunogen Organism Species,Immunogen Organism Species Accession,In Vitro Process Type,Responder Cell Type,Stimulator Cell Type,Immunogen Epitope Relation,Immunogen Object Type,Immunogen Description,Non-peptidic Immunogen ChEBI ID,Immunogen Source Molecule Name,Protein Parent Name,Protein Parent Accession,Immunogen Organism Name,Immunogen Organism Species,Immunogen Organism Species ID,Adoptive Transfer Flag,Adoptive Transfer Comments,Immunization Comments,Location of assay data in the manuscript,Method/Technique,Assay Group,Units,Qualitative Measure,Measurement Inequality,Quantitative measurement,Number of Subjects Tested,Number of Subjects Responded,Response Frequency,Cell Tissue Type,Cell Type,Cell Culture Conditions,Assayed TCR Molecule Name,PDB ID,Cell Tissue Type,Cell Type,Cell Culture Conditions,Allele Name,Class,Allele Evidence Code,Antigen Epitope Relation,Antigen Object Type,Antigen Description,Starting Position,Ending Position,Non-peptidic Antigen ChEBI ID,Antigen Source Molecule Name,Protein Parent Name,Protein Parent Accession,Antigen Organism Name,Organism Species Name,Organism Species ID,Assay Comments
1,73,284,Literature,15694996,Corinna La Rosa; Zhongde Wang; Simon F Lacey; ...,Hum Immunol,2005,Characterization of host immunity to cytomegal...,NaN,66913,Linear peptide,TTVYPPSSTAK,945,955,NaN,phosphoprotein 150,Tegument protein pp150,Q6SW99,Human betaherpesvirus 5,Human betaherpesvirus 5,10359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homo sapiens,9606,NaN,NaN,Exposure with existing immune reactivity witho...,NaN,NaN,Source Organism,Organism,Human betaherpesvirus 5,NaN,NaN,NaN,NaN,NaN,NaN,Human betaherpesvirus 5,Human betaherpesvirus 5,10359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Restimulation in vitro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,HLA restricted pp150 CTL clone was derived fro...,Figure 6,multimer/tetramer,qualitative binding,NaN,Positive,NaN,NaN,NaN,NaN,NaN,Blood,T cell CD8+,Cell Line / Clone,UPN 002 clone 5,NaN,NaN,NaN,NaN,HLA-A*03:01,I,T cell assay -Mismatched MHC molecules,Epitope,Linear peptide,TTVYPPSSTAK,945,955,NaN,phosphoprotein 150,Tegument protein pp150,Q6SW99,Human betaherpesvirus 5,Human betaherpesvirus 5,10359,Overlapping peptides were tested to determine ...
2,105,274,Literature,15448372,Yi-Hsiang Huang; Mi-Hua Tao; Cheng-po Hu; Wan-...,J Gen Virol,2004,Identification of novel HLA-A*0201-restricted ...,NaN,31803,Linear peptide,KLEDLERDL,26,34,NaN,large delta antigen,Large delta antigen,P29996,Hepatitis delta virus TW2667,Hepatitis delta virus,12475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homo sapiens,9606,NaN,NaN,Occurrence of infectious disease,hepatitis D,Chronic,Taxonomic Parent,Organism,Hepatitis delta virus,NaN,NaN,NaN,NaN,NaN,NaN,Hepatitis de

In [3]:
# Extracting TCRs
# https://www.iedb.org/assay/"T Cell ID" <- webpage for each of these
baseURL = 'https://www.iedb.org/assay/'
proteinURLs = {}

for tId in listofIds:
    try:
        r = requests.get(baseURL + tId)
        soup = BeautifulSoup(r.content,'lxml')
        pattern = 'var resultData.*};'
        a = re.search(pattern, str(soup))
        extractedJson = re.sub('^var resultData = ', '', a.group())
        epitope = json.loads(extractedJson[:-1])['ant_mol1_seq']['value']
        proteinURLs[epitope] = json.loads(extractedJson[:-1])['tcr_mol1_name']['identifier'].replace('GenPept:', '')
    except:
        pass

In [4]:
# Find proteins on NCBI
baseURL = 'https://www.ncbi.nlm.nih.gov/protein/'
for identifier in proteinURLs:
    proteinURLs[identifier] = baseURL + proteinURLs[identifier]
    # browser = webdriver.Chrome()
    # browser.delete_all_cookies()
    # browser.get(baseURL + proteinURLs[identifier])
    # link_tags = browser.find_elements_by_tag_name('span')
proteinURLs

{'LPEPLPQGQLTAY': 'https://www.ncbi.nlm.nih.gov/protein/2AK4_T',
 'SLLMWITQC': 'https://www.ncbi.nlm.nih.gov/protein/2F53_D',
 'SLLMWITQV': 'https://www.ncbi.nlm.nih.gov/protein/2F53_D',
 'GELIGTLNAAKVPAD': 'https://www.ncbi.nlm.nih.gov/protein/2IAN_D',
 'CLGGLLTMV': 'https://www.ncbi.nlm.nih.gov/protein/3REV_A',
 'FATGIGIITV': 'https://www.ncbi.nlm.nih.gov/protein/3HG1_D',
 'IQGNVTSIHSLLDEG': 'https://www.ncbi.nlm.nih.gov/protein/CCI73895.1',
 'IHSLLDEGKQSLTKL': 'https://www.ncbi.nlm.nih.gov/protein/CCI73895.1',
 'AAVVRFQEAANKQK': 'https://www.ncbi.nlm.nih.gov/protein/CCI73895.1',
 'ELAGIGILTV': 'https://www.ncbi.nlm.nih.gov/protein/6D7G_D',
 'FLAGGIGIRTL': 'https://www.ncbi.nlm.nih.gov/protein/3HG1_D',
 'FFAGGIGIRTI': 'https://www.ncbi.nlm.nih.gov/protein/3HG1_D',
 'FWLLGAWAL': 'https://www.ncbi.nlm.nih.gov/protein/3HG1_D',
 'FWLLPAWAL': 'https://www.ncbi.nlm.nih.gov/protein/3HG1_D',
 'WLLPAWGV': 'https://www.ncbi.nlm.nih.gov/protein/3HG1_D',
 'WLLPTWGV': 'https://www.ncbi.nlm.nih.go

In [6]:
import csv
with open('iedb_data_filtered.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in proteinURLs.items():
       writer.writerow([key, value])